In [ ]:
# default_exp main

# rfpy - CFRS Rfeye Node Logger binaries parser

In [ ]:
#hide
%load_ext autoreload
%autoreload 2            #Reload the code automatically

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#export
from typing import *
from fastcore.xtras import Path
from fastcore.script import *
from rfpy.parser import *
import pandas as pd
from tqdm import tqdm 

In [ ]:
#export 
def _export_meta(bin_path: Union[str, Path],
                bin_block: DefaultDict,
                saida: Union[str, Path], 
                ext: str = '.csv',
                return_df: bool = False)->Union[None, pd.DataFrame]:
    meta_items = extract_metadata(bin_block)
    for (tipo, tid), df in meta_items:
        dt_features = isinstance(df.index, pd.DatetimeIndex)
        if dt_features:
            start = df.index.min().strftime("%Y%m%d-%H%M%S")
            end = df.index.max().strftime("%Y%m%d-%H%M%S")
            name = f'{bin_path.stem}_Bloco_{tipo}_ThreadId_{tid}_{start}_{end}'
        else:
            name = f'{bin_path.stem}_Bloco_{tipo}_ThreadId_{tid}'
        if ext == '.csv':
            df.to_csv(Path(saida) / f'{name}{ext}', index=bool(dt_features))
        elif ext == '.fth':
            df.reset_index().to_feather(Path(saida) / f'{name}{ext}')
        else:
            raise ValueError(f"Extension {ext} not implemented")
        if return_df:
            return df

In [ ]:
#export
@call_parse
def export_metadata(entrada:Param("Diretório contendo arquivos .bin", str),
                saida:Param("Diretório para salvar os arquivos de saída", str),
                recursivo:Param("Buscar arquivos de maneira recursiva?", bool_arg)=True,
                pastas:Param("Limita a busca às pastas", Iterable[str]) = None,
                ext:Param("Qual extensão salvar os arquivos", str) = '.csv',
                verbose:Param("Imprimir mensagens de execução?", bool_arg) = False):

        lista_bins = get_files(entrada, extensions=['.bin'], recurse=recursivo, folders=pastas)
        bin_blocks = lista_bins.map(parse_bin)
        if verbose: 
            print(lista_bins)
        for bin_path, bin_block in tqdm(zip(lista_bins, bin_blocks), total=len(lista_bins)):
            _export_meta(bin_path, bin_block, saida, ext)

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_main.ipynb.
Converted 01_parser.ipynb.
Converted 02_utils.ipynb.
Converted 03_blocks.ipynb.
Converted 04_constants.ipynb.
